In [1]:
import numpy as np
import os
import torch
import pandas as pd
import yaml

from sklearn.externals import joblib

# from pytorch_utils.models import FeedforwardNetModel
from pytorch_utils.cfvae_models import CFVAEModel

In [2]:
outcome = 'los'
sensitive_variable = 'age'
data_path = 'data/'

# features_path = os.path.join(data_path, '{}_excluded'.format(sensitive_attibute),'features', str(0))
features_path = os.path.join(data_path, 'features', str(0))
label_path = os.path.join(data_path, 'labels')
config_path = os.path.join(data_path, 'config', 'grid', 'baseline')
checkpoints_path = os.path.join(data_path, 'checkpoints', 'scratch', outcome)
performance_path = os.path.join(data_path, 'performance', 'scratch', outcome)

In [3]:
os.makedirs(checkpoints_path, exist_ok=True)
os.makedirs(performance_path, exist_ok=True)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
features_dict = joblib.load(os.path.join(features_path, 'features.pkl'))
master_label_dict = joblib.load(os.path.join(label_path, 'label_dict.pkl'))

In [7]:
data_dict = {split: features_dict[split]['features'] for split in features_dict.keys()}
label_dict = {split : master_label_dict[split][outcome] for split in master_label_dict.keys()}
group_dict = {split : master_label_dict[split][sensitive_variable] for split in master_label_dict.keys()}

In [8]:
group_map = pd.read_csv(os.path.join(label_path, '{}_map.csv'.format(sensitive_variable)))

In [9]:
# # with open(os.path.join(config_path, '{}.yaml'.format(grid_element)), 'r') as fp:
# #     config_dict = yaml.load(fp)
    
# # config_dict['num_epochs'] = 3 # For testing

# ## A more complex network
# # config_dict = {
# #     'input_dim' : data_dict['train'].shape[1],
# #     'lr' : 1e-5,
# #     'num_epochs' : 20,
# #     'batch_size' : 256,
# #     'hidden_dim' : 128,
# #     'num_hidden' : 1,
# #     'output_dim' : 2,
# #     'drop_prob' : 0.5,
# #     'normalize' : True,
# #     'iters_per_epoch' : 100,
# #     'gamma' : 0.99,
# #     'resnet' : True,
# #     'sparse' : True,
# #     'sparse_mode' : 'binary'
# # }

# CFVAE
config_dict = {
    # Standard parameters
    'input_dim' : data_dict['train'].shape[1],
    'num_groups' : group_map.shape[0],
    'lr' : 1e-3,
    'gamma' : 0.99,
    'num_epochs' : 30,
    'iters_per_epoch' : 100,
    'output_dim' : 2,
    'batch_size' : 256,
    'sparse' : True,
    'sparse_mode' : 'binary',
    # Parameters corresponding to the size of the VAE
    'hidden_dim' : 128,
    'num_hidden' : 1,
    'drop_prob' : 0.0,
    'resnet' : False,
    'normalize' : False,
    # Parameters corresponding to the size of classifier
    'hidden_dim_classifier' : 128,
    'num_hidden_classifier' : 1,
    'drop_prob_classifier' : 0.0,
    'resnet_classifier' : False,
    'normalize_classifier' : False,
    # Latent Space
    'group_embed_dim' : 64,
    'latent_dim' : 64,
    # Lambda
    'lambda_mmd' : 1e3,
    'lambda_kl' : 0.0,
    'lambda_classification' : 1e2,
    'lambda_mmd_group' : 1e0
}

In [10]:
model = CFVAEModel(config_dict)

In [12]:
%%time
result = model.train(data_dict, label_dict, group_dict)

Epoch 0/19
----------
Phase: train:
 loss: 44.418334, elbo: 15.803110, mmd: 0.001215, reconstruction: 0.179762, kl: 15.193207, classification: 0.430141, mmd_group: 0.009696,
 auc: 0.770880, auprc: 0.428378, brier: 0.139533,
Phase: val:
 loss: 40.145067, elbo: 24.268931, mmd: 0.000616, reconstruction: 0.007639, kl: 23.866174, classification: 0.395118, mmd_group: 0.009928,
 auc: 0.817798, auprc: 0.507240, brier: 0.126629,
Best model updated
Epoch 1/19
----------
Phase: train:
 loss: 36.265467, elbo: 27.254481, mmd: 0.000482, reconstruction: 0.005301, kl: 26.891496, classification: 0.357685, mmd_group: 0.009612,
 auc: 0.854497, auprc: 0.622286, brier: 0.113240,
Phase: val:
 loss: 38.803118, elbo: 27.765742, mmd: 0.000384, reconstruction: 0.004071, kl: 27.377595, classification: 0.384076, mmd_group: 0.007795,
 auc: 0.825927, auprc: 0.543091, brier: 0.123493,
Best model updated
Epoch 2/19
----------
Phase: train:
 loss: 33.117015, elbo: 34.879278, mmd: 0.000510, reconstruction: 0.003609, kl

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/labs/shahlab/spfohl/miniconda3/envs/py_env/lib/python3.6/site-packages/IPython/core/magics/execution.py", line 1238, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 1, in <module>
  File "/home/spfohl/projects/fairness_cf/pytorch_utils/cfvae_models.py", line 243, in train
    target = torch.FloatTensor(inputs.todense()).to(self.device)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/labs/shahlab/spfohl/miniconda3/envs/py_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/labs/shahlab/spfohl/miniconda3/envs/py_env/lib/python3.6/site-packages/IPython/core/ultratb.py", line

KeyboardInterrupt: 

In [13]:
# result_final_classifier = model.train(data_dict, label_dict, group_dict)
result_eval = model.predict(data_dict, label_dict, group_dict, phases = ['val', 'test'])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/labs/shahlab/spfohl/miniconda3/envs/py_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-637ddb16782a>", line 2, in <module>
    result_eval = model.predict(data_dict, label_dict, group_dict, phases = ['val', 'test'])
TypeError: predict() got multiple values for argument 'phases'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/labs/shahlab/spfohl/miniconda3/envs/py_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/labs/shahlab/spfohl/miniconda3/envs/py_env/lib/python3.6/site-packages/IPython/core

TypeError: predict() got multiple values for argument 'phases'

In [ ]:
## Save weights
# model.save_weights(os.path.join(checkpoints_path, '{}.chk'.format(grid_element)))